In [ ]:
# default_exp cdx

# CDX

This is an interface to the [Common Crawl Index](http://commoncrawl.org/2015/04/announcing-the-common-crawl-index/) and the [Internet Archive's CDX Server](https://github.com/internetarchive/wayback/tree/master/wayback-cdx-server) for finding archived resources based on URL patterns.
It has a lot of overlap with [cdx_toolkit](https://github.com/cocrawler/cdx_toolkit) but is more optimised for speed and ease of use.

In [ ]:
#export
# Typing
from __future__ import annotations # For Python <3.9
from typing import Any, Optional, Union
from collections.abc import Iterable

import requests
from requests.sessions import Session
import json

In [ ]:
test_cache = 'data/test/cache'

# Internet Archive


## Querying CDX

The Internet Archive runs it's own [Java CDX Server](https://github.com/internetarchive/wayback/tree/master/wayback-cdx-server). [Hakernoon has an article](https://hackernoon.com/guide-to-handling-internet-archives-cdx-server-api-response-c469df5b81f4) that gives a good overview of querying it.

See the documentation for all the paramemters; here are the ones that I find most interesting

* url: The URL to query, a wildcard `*` can implicitly define it (which means we don't need `matchType`)
* output: "json" returns JSON instead of space separated
* fl: Comma separated list of fields to return
* from, to: Date filtering 1-14 digits yyyyMMddhhmmss, inclusive (this seems to do the right thing for truncation).
* filter: `[!]field:regex` filter a returned field with [Java Regex](https://docs.oracle.com/javase/6/docs/api/java/util/regex/Pattern.html), inverting with `!`
* limit/offset: For getting a small number of results (sampling). Internal limit is 150000 results.

Default fields returned: ["urlkey","timestamp","original","mimetype","statuscode","digest","length"]

Pagination isn't really useful because it's applied *before* filtering (including date filtering) so most pages are empty with a date filter.
Unpaginated requests are fast enough anyway.

### Exporting the data

The data is a JSON array or arrays; the first line contains the headers and the subsequent the data.
Let's transform it into a list of dictionaries of keys to values which make it a bit easier to work with (although less efficient in memory).

An alternative would be to directly use something like Pandas

In [ ]:
#export
def header_and_rows_to_dict(rows: Iterable[list[Any]]) -> list[dict[Any, Any]]:
    header = None
    data = []
    for row in rows:
        if header is None:
            header = row
        else:
            assert len(row) == len(header), "Row should be same length as header"
            data.append(dict(zip(header, row)))
    return data

Check it on some data

In [ ]:
assert header_and_rows_to_dict([['col_1', 'col_2'], [1, 'a'], [2, 'b']]) == [
    {'col_1': 1, 'col_2': 'a'},
    {'col_1': 2, 'col_2': 'b'}]

In [ ]:
assert header_and_rows_to_dict([['col_1', 'col_2']]) == []

In [ ]:
assert header_and_rows_to_dict([]) == []

### Making a query

In [ ]:
#export
IA_CDX_URL = 'http://web.archive.org/cdx/search/cdx'

# This could be more precise
CaptureIndexRecord = dict

def mimetypes_to_regex(mime: list[str], prefix='mimetype:') -> str:
    return prefix + '|'.join('(' + s.replace('*', '.*') + ')' for s in mime)

def query_wayback_cdx(url: str, start: Optional[str], end: Optional[str],
                      status_ok: bool = True, 
                      mime: Optional[Union[str, Iterable[str]]] = None,
                      limit: Optional[int] = None, offset: Optional[int] = None,
                      session: Optional[Session] = None) -> list[CaptureIndexRecord]:
    """Get references to Wayback Machine Captures for url.
    
    Queries the Internet Archive Capture Index (CDX) for url.
    
    Arguments:
      * start: Minimum date in format YYYYmmddHHMMSS (or any substring) inclusive
      * end: Maximum date in format YYYYmmddHHMMSS (or any substring) inclusive
      * status_ok: Only return those with a HTTP status 200
      * mime: Filter on mimetypes, '*' is a wildcard (e.g. 'image/*')
      * limit: Only return first limit records
      * offset: Skip the first offset records, combine with limit
      * session: Session to use when making requests
    Filters results between start and end inclusive, in format YYYYmmddHHMMSS or any substring
    (e.g. start="202001", end="202001" will get all captures in January 2020)
    """
    if session is None:
        session = requests
        
    params = {'url': url,
              'output': 'json',
              'from': start,
              'to': end,
              'limit': limit,
              'offset': offset}
    
    filter = []
    if status_ok:
        filter.append('statuscode:200')
    if mime:
        # Turn our list of mimetypes into a regex
        if isinstance(mime, str):
            mime = [mime]
        filter.append(mimetypes_to_regex(mime))
    params['filter'] = filter
    
    params = {k:v for k,v in params.items() if v}
    response = session.get(IA_CDX_URL, params=params)
    response.raise_for_status()
    return header_and_rows_to_dict(response.json())

#### Test querying

In [ ]:
%%time
full_sample = query_wayback_cdx('skeptric.com/*', start=None, end=None, status_ok=False)

CPU times: user 2.83 ms, sys: 9.67 ms, total: 12.5 ms
Wall time: 875 ms


In [ ]:
assert len(full_sample) > 1000
len(full_sample)

1579

In [ ]:
import pandas as pd
pd.DataFrame(full_sample)

,urlkey,timestamp,original,mimetype,statuscode,digest,length
0,"com,skeptric)/",20180805132731,http://skeptric.com/,text/html,200,2N5QQYZFAM36CSESTGSDKRTRV7I5HEXJ,1982
1,"com,skeptric)/",20190610142553,http://skeptric.com/,warc/revisit,-,2N5QQYZFAM36CSESTGSDKRTRV7I5HEXJ,580
2,"com,skeptric)/",20201126064102,https://skeptric.com/,text/html,200,WDYU3RU7ZMFFSZPAPE56PC4L3EK4FE3D,82985
3,"com,skeptric)/",20210109075617,https://skeptric.com/,text/html,200,M6GZ4ZVD7U5L2TUXS5POHN36SSLOEPO7,92869
4,"com,skeptric)/",20210417050827,https://skeptric.com/,text/html,200,ZOYZF2RVY44PBSNNXMVH27MHUPGGO4OK,101114
...,...,...,...,...,...,...,...
1574,"com,skeptric)/wsl2-xserver",20201216173655,https://skeptric.com/wsl2-xserver/,warc/revisit,-,UAS4RFHUDI6TNMPWVTD4N6RZWLKQAOGT,720
1575,"com,skeptric)/wsl2-xserver",20201216185608,https://skeptric.com/wsl2-xserver/,warc/revisit,-,UAS4RFHUDI6TNMPWVTD4N6RZWLKQAOGT,721
1576,"com,skeptric)/wsl2-xserver",20210128160531,https://skeptric.com/wsl2-xserver/,text/html,200,BBSHS372N5MIYDLGSQWQKF6TKNBEDD35,5794
1577,"com,skeptric)/wsl2-xserver",20210520180504,https://skeptric.com/wsl2-xserver/,text/html,200,BBSHS372N5MIYDLGSQWQKF6TKNBEDD35,4821


In [ ]:
timestamps = [x['timestamp'] for x in full_sample]
min(timestamps), max(timestamps)

('20170915060403', '20211123083615')

In [ ]:
pd.DataFrame(full_sample).groupby(['mimetype', 'statuscode'])['urlkey'].count()

mimetype                  statuscode
application/javascript    200             1
application/octet-stream  200             3
image/gif                 200             3
image/jpeg                200            59
image/png                 200           292
image/svg+xml             200            51
image/vnd.microsoft.icon  200             1
image/x-icon              200             2
text/css                  200            10
text/html                 200           393
                          302             1
                          404             5
warc/revisit              -             758
Name: urlkey, dtype: int64

#### Test statuscode filter

In [ ]:
%%time
ok_sample = query_wayback_cdx('skeptric.com/*', start=None, end=None) #status_ok=True

CPU times: user 7.22 ms, sys: 3.09 ms, total: 10.3 ms
Wall time: 659 ms


In [ ]:
assert ok_sample == [x for x in full_sample if x['statuscode'] == '200']

#### Test mimetypes

In [ ]:
%%time
html_sample = query_wayback_cdx('skeptric.com/*', start=None, end=None, status_ok=False, mime='text/html')

CPU times: user 12 ms, sys: 735 µs, total: 12.7 ms
Wall time: 676 ms


In [ ]:
assert html_sample == [x for x in full_sample if x['mimetype'] == 'text/html']

In [ ]:
%%time
image_sample = query_wayback_cdx('skeptric.com/*', start=None, end=None, status_ok=False, mime='image/*')

CPU times: user 7.9 ms, sys: 0 ns, total: 7.9 ms
Wall time: 669 ms


In [ ]:
assert image_sample == [x for x in full_sample if x['mimetype'].startswith('image/')]

In [ ]:
%%time
prog_sample = query_wayback_cdx('skeptric.com/*', start=None, end=None, status_ok=False, 
                                 mime=['text/css', 'application/javascript'])

CPU times: user 0 ns, sys: 6.06 ms, total: 6.06 ms
Wall time: 467 ms


In [ ]:
assert prog_sample == [x for x in full_sample if x['mimetype'] in ['text/css', 'application/javascript']]

#### Test date filters

In [ ]:
%%time
sample_2020 = query_wayback_cdx('skeptric.com/*', start='2020', end='2020', status_ok=False)
len(sample_2020)

CPU times: user 4.91 ms, sys: 2.1 ms, total: 7.01 ms
Wall time: 781 ms


106

In [ ]:
assert sample_2020 == [x for x in full_sample if '2020' <= x['timestamp'] < '2021']

In [ ]:
%%time
sample_to_2020 = query_wayback_cdx('skeptric.com/*', start=None, end='2020', status_ok=False)

CPU times: user 3.64 ms, sys: 1.09 ms, total: 4.73 ms
Wall time: 453 ms


In [ ]:
assert sample_to_2020 == [x for x in full_sample if x['timestamp'] < '2021']

In [ ]:
%%time
sample_from_2020 = query_wayback_cdx('skeptric.com/*', start='2020', end=None, status_ok=False)

CPU times: user 12.7 ms, sys: 945 µs, total: 13.6 ms
Wall time: 841 ms


In [ ]:
assert sample_from_2020 == [x for x in full_sample if x['timestamp'] >= '2020']

#### Test limits

In [ ]:
%%time
sample_10 = query_wayback_cdx('skeptric.com/*', start=None, end=None, status_ok=False, limit=10)

CPU times: user 3.19 ms, sys: 1.3 ms, total: 4.48 ms
Wall time: 443 ms


In [ ]:
sample_10 == full_sample[:10]

True

In [ ]:
%%time
sample_10_offset_20 = query_wayback_cdx('skeptric.com/*', start=None, end=None, status_ok=False, limit=10, offset=20)

CPU times: user 4.32 ms, sys: 1.77 ms, total: 6.1 ms
Wall time: 497 ms


In [ ]:
assert sample_10_offset_20  == full_sample[20:20+10]

## Fetching Content

In [ ]:
#export
def fetch_internet_archive_content(timestamp: str, url: str,
                                   session: Optional[Session] = None) -> bytes:
    if session is None:
        session = requests
        
    url = f'http://web.archive.org/web/{timestamp}/{url}'
    response = session.get(url)
    # Sometimes Internet Archive deletes records
    if response.status_code == 404:
        logging.warning(f'Missing {url}')
        return None
    response.raise_for_status()
    return response.content

In [ ]:
record = image_sample[0]
record

{'urlkey': 'com,skeptric)/favicon.ico',
 'timestamp': '20180819051126',
 'original': 'http://skeptric.com/favicon.ico',
 'mimetype': 'image/vnd.microsoft.icon',
 'statuscode': '200',
 'digest': 'R6YE2GPPT4BM4IAMHGUJPDJF6BGKKHDA',
 'length': '851'}

In [ ]:
content = fetch_internet_archive_content(record['timestamp'], record['original'])

In [ ]:
from IPython.display import Image
Image(content)

In [ ]:
len(content)

1150

In [ ]:
record = ok_sample[0]
record

{'urlkey': 'com,skeptric)/',
 'timestamp': '20180805132731',
 'original': 'http://skeptric.com/',
 'mimetype': 'text/html',
 'statuscode': '200',
 'digest': '2N5QQYZFAM36CSESTGSDKRTRV7I5HEXJ',
 'length': '1982'}

In [ ]:
content = fetch_internet_archive_content(record['timestamp'], record['original'])

In [ ]:
content[:100]

b'<!DOCTYPE html>\n<html lang="en">\n  <head><script src="//archive.org/includes/analytics.js?v=cf34f82"'

## Wrap it in a fast Object

I have no idea whether Session is [actually thread-safe](https://github.com/urllib3/urllib3/issues/1252); if it's not we should have 1 session per thread. As far as I can tell the issue occurs when you have lots of hosts, so in this case it should be ok.

I guess we'll try it and see. Maybe long term we're better going with asyncio.
Using a Session makes things slightly faster; we can always turn it off by passing session=False.

In [ ]:
#export
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

def make_session(pool_maxsize):
    retry_strategy =  Retry(total=5, backoff_factor=1, status_forcelist=set([504, 500]))
    adapter = HTTPAdapter(max_retries=retry_strategy, pool_maxsize=pool_maxsize, pool_block=True)
    session = requests.Session()
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

We cache everything with joblib and make it fast.

Note we only use 8 threads to avoid overloading the servers.

In [ ]:
#export
from joblib import delayed, Memory, Parallel

class WaybackResult:
    def __init__(self, params, session=None):
        self.params = params
        self.session = session
        self._query = self.memory.cache(query_wayback_cdx, ignore=['session']) 
    
    def fetch(self, force=False):
        fetch = self._fetch.call if force else self._fetch
        result = fetch(record['timestamp'], record['original'], session=self.session)
        return result[0] if force else result
        

class WaybackMachine:
    def __init__(self, location: Optional[str],
                 session: Union[bool, Session] = True, threads: int=8, verbose=0) -> None:
        self.threads = threads
        self.memory = Memory(location, verbose=verbose)
        
        if session is True:
            session = make_session(pool_maxsize=threads)
        if not session:
            session = None
        self.session = session
        
        self._query = self.memory.cache(query_wayback_cdx, ignore=['session'])
        self._fetch = self.memory.cache(fetch_internet_archive_content, ignore=['session'])
        
    def query(self, url: str, start: Optional[str], end: Optional[str],
                      status_ok: bool = True, mime: Optional[Union[str, Iterable[str]]] = None,
                      limit: Optional[int] = None, offset: Optional[int] = None, force=False) -> List[CaptureIndexRecord]:
        query = self._query.call if force else self._query
        result = query(url, start, end, status_ok, mime, limit, offset, session=self.session)
        return result[0] if force else result
    
    def fetch_one(self, record, force=False) -> bytes:
        fetch = self._fetch.call if force else self._fetch
        result = fetch(record['timestamp'], record['original'], session=self.session)
        return result[0] if force else result
    
    def fetch(self, records, force=False, threads=None) -> bytes:
        fetch = self._fetch.call if force else self._fetch
        result = Parallel(threads or self.threads, prefer='threads')(delayed(self.fetch_one)(record, force) for record in records)
        return result

#### Test fetching one

In [ ]:
wb = WaybackMachine(test_cache, verbose=1)

In [ ]:
wb.memory.clear()

In [ ]:
%%time
items = wb.query('skeptric.com/*', None, None)

________________________________________________________________________________
[Memory] Calling __main__--tmp-ipykernel-2773891608.query_wayback_cdx...
query_wayback_cdx('skeptric.com/*', None, None, True, None, None, None, session=<requests.sessions.Session object at 0x7f06ad067be0>)
________________________________________________query_wayback_cdx - 0.9s, 0.0min
CPU times: user 62.1 ms, sys: 0 ns, total: 62.1 ms
Wall time: 881 ms


In [ ]:
%%time
items2 = wb.query('skeptric.com/*', None, None)

CPU times: user 22.9 ms, sys: 0 ns, total: 22.9 ms
Wall time: 19.6 ms


In [ ]:
assert items2 == items

In [ ]:
%%time
items3 = wb.query('skeptric.com/*', None, None, force=True)

________________________________________________________________________________
[Memory] Calling __main__--tmp-ipykernel-2773891608.query_wayback_cdx...
query_wayback_cdx('skeptric.com/*', None, None, True, None, None, None, session=<requests.sessions.Session object at 0x7f06ad067be0>)
________________________________________________query_wayback_cdx - 0.7s, 0.0min
CPU times: user 39.1 ms, sys: 3.69 ms, total: 42.8 ms
Wall time: 682 ms


In [ ]:
assert items3 == items

In [ ]:
%%time
content = wb.fetch_one(items[0])

________________________________________________________________________________
[Memory] Calling __main__--tmp-ipykernel-2187831798.fetch_internet_archive_content...
fetch_internet_archive_content('20180805132731', 'http://skeptric.com/', session=<requests.sessions.Session object at 0x7f06ad067be0>)
___________________________________fetch_internet_archive_content - 0.2s, 0.0min
CPU times: user 8.33 ms, sys: 0 ns, total: 8.33 ms
Wall time: 179 ms


In [ ]:
%%time
content2 = wb.fetch_one(items[0])

CPU times: user 3.11 ms, sys: 0 ns, total: 3.11 ms
Wall time: 1.73 ms


In [ ]:
assert content2 == content

In [ ]:
%%time
content3 = wb.fetch_one(items[0], force=True)

________________________________________________________________________________
[Memory] Calling __main__--tmp-ipykernel-2187831798.fetch_internet_archive_content...
fetch_internet_archive_content('20180805132731', 'http://skeptric.com/', session=<requests.sessions.Session object at 0x7f06ad067be0>)
___________________________________fetch_internet_archive_content - 0.2s, 0.0min
CPU times: user 7.05 ms, sys: 1.43 ms, total: 8.48 ms
Wall time: 184 ms


In [ ]:
assert content3 == content

#### Test fetching many

In [ ]:
wb = WaybackMachine(test_cache, verbose=0)

In [ ]:
%%time
contents_16 = wb.fetch(items[:16], threads=8)

CPU times: user 168 ms, sys: 57.3 ms, total: 226 ms
Wall time: 3.09 s


In [ ]:
assert contents_16[0] == content

# Common Crawl

Common Crawl is similar, but slightly different, because it is split over many indexes, roughly monthly covering 2 weeks.

## Indexes

Cache within a session, but not between sessions since over months the indexes change.

In [ ]:
#export
from functools import lru_cache
@lru_cache(maxsize=None)
def get_cc_indexes() -> List[Dict[str, str]]:
    response = requests.get("https://index.commoncrawl.org/collinfo.json")
    response.raise_for_status()
    return response.json()

In [ ]:
indexes = get_cc_indexes()
indexes[:3]

[{'id': 'CC-MAIN-2021-43',
  'name': 'October 2021 Index',
  'timegate': 'https://index.commoncrawl.org/CC-MAIN-2021-43/',
  'cdx-api': 'https://index.commoncrawl.org/CC-MAIN-2021-43-index'},
 {'id': 'CC-MAIN-2021-39',
  'name': 'September 2021 Index',
  'timegate': 'https://index.commoncrawl.org/CC-MAIN-2021-39/',
  'cdx-api': 'https://index.commoncrawl.org/CC-MAIN-2021-39-index'},
 {'id': 'CC-MAIN-2021-31',
  'name': 'July 2021 Index',
  'timegate': 'https://index.commoncrawl.org/CC-MAIN-2021-31/',
  'cdx-api': 'https://index.commoncrawl.org/CC-MAIN-2021-31-index'}]

Notice the oldest ones have a different format

In [ ]:
indexes[-4:]

[{'id': 'CC-MAIN-2013-20',
  'name': 'Summer 2013 Index',
  'timegate': 'https://index.commoncrawl.org/CC-MAIN-2013-20/',
  'cdx-api': 'https://index.commoncrawl.org/CC-MAIN-2013-20-index'},
 {'id': 'CC-MAIN-2012',
  'name': 'Index of 2012 ARC files',
  'timegate': 'https://index.commoncrawl.org/CC-MAIN-2012/',
  'cdx-api': 'https://index.commoncrawl.org/CC-MAIN-2012-index'},
 {'id': 'CC-MAIN-2009-2010',
  'name': 'Index of 2009 - 2010 ARC files',
  'timegate': 'https://index.commoncrawl.org/CC-MAIN-2009-2010/',
  'cdx-api': 'https://index.commoncrawl.org/CC-MAIN-2009-2010-index'},
 {'id': 'CC-MAIN-2008-2009',
  'name': 'Index of 2008 - 2009 ARC files',
  'timegate': 'https://index.commoncrawl.org/CC-MAIN-2008-2009/',
  'cdx-api': 'https://index.commoncrawl.org/CC-MAIN-2008-2009-index'}]

## Parsing the times

In [ ]:
import re
from datetime import datetime
year_week = re.match(r'^CC-MAIN-(\d{4}-\d{2})$', indexes[0]['id']).group(1)
datetime.strptime(year_week  + '-6', '%Y-%W-%w')

datetime.datetime(2021, 10, 30, 0, 0)

In [ ]:
indexes[0]['id']

'CC-MAIN-2021-43'

## Querying CDX

https://pywb.readthedocs.io/en/latest/manual/cdxserver_api.html

http://commoncrawl.org/2015/04/announcing-the-common-crawl-index/

In [ ]:
cc_sample_api = 'https://index.commoncrawl.org/CC-MAIN-2021-43-index'
assert cc_sample_api in [i['cdx-api'] for i in indexes]

The Common Crawl CDX is *much slower* than the Wayback Machines.
It also uses the zipped CDX by default which automatically paginates.

In [ ]:
#export
import json

def jsonl_loads(jsonl):
    return [json.loads(line) for line in jsonl.splitlines()]

In [ ]:
test_jline = b'{"status": "200", "mime": "text/html"}\n{"status": "301", "mime": "-"}\n'
test_jdict = [{'status': '200', 'mime': 'text/html'}, {'status': '301', 'mime': '-'}]
assert jsonl_loads(test_jline) == test_jdict
assert jsonl_loads(test_jline.rstrip()) == test_jdict

In [ ]:
#export
def query_cc_cdx_num_pages(api: str,  url: str,
                           session: Optional[Session] = None) -> int:
    if session is None:
        session = requests
        
    response = session.get(api, params={'url': url, 'output': 'json', 'showNumPages': True})
    response.raise_for_status()
    data = response.json()
    return data["pages"]

def query_cc_cdx_page(
                 api: str, url: str, page: int, 
                 start: Optional[str] = None, end: Optional[str] = None,
                 status_ok: bool = True, mime: Optional[Union[str, Iterable[str]]] = None,
                 limit: Optional[int] = None, offset: Optional[int] = None,
                 session: Optional[Session] = None) -> List[CaptureIndexRecord]:
    """Get references to Common Crawl Captures for url.
    
    Queries the Common Crawl Capture Index (CDX) for url.
    
    Filters:
      * api: API endpoint to use (e.g. 'https://index.commoncrawl.org/CC-MAIN-2021-43-index')
      * start: Minimum date in format YYYYmmddHHMMSS (or any substring) inclusive
      * end: Maximum date in format YYYYmmddHHMMSS (or any substring) inclusive
      * status_ok: Only return those with a HTTP status 200
      * mime: Filter on mimetypes, '*' is a wildcard (e.g. 'image/*')
      * limit: Only return first limit records
      * offset: Skip the first offset records, combine with limit
      * session: Session to use when making requests
    Filters results between start and end inclusive, in format YYYYmmddHHMMSS or any substring
    (e.g. start="202001", end="202001" will get all captures in January 2020)
    """
    if session is None:
        session = requests
        
    params = {'url': url,
              'page': page,
              'output': 'json',
              'page': page,
              'from': start,
              'to': end,
              'limit': limit,
              'offset': offset}
    
    filter = []
    if status_ok:
        # N.B. Different to IA
        filter.append('=status:200')
    if mime:
        if isinstance(mime, str):
            mime = [mime]
        # N.B. Different to IA
        filter.append(mimetypes_to_regex(mime, prefix='~mime:'))
    params['filter'] = filter
    
    params = {k:v for k,v in params.items() if v}
    response = session.get(api, params=params)
    response.raise_for_status()
    return jsonl_loads(response.content)

### Testing pagination

In [ ]:
test_url = 'mn.wikipedia.org/*'

In [ ]:
#slow
pages = query_cc_cdx_num_pages(cc_sample_api, test_url)
pages

2

In [ ]:
#slow
records = []
for page in range(pages):
    records += query_cc_cdx_page(cc_sample_api, test_url, page=page, mime='text/html')
len(records)

17384

In [ ]:
#slow
timestamps = [record['timestamp'] for record in records]
min(timestamps), max(timestamps)

('20211015193030', '20211028220724')

We should get an error querying past the last page (400: Invalid Request)

In [ ]:
#slow
from requests.exceptions import HTTPError

try:
    query_cc_cdx_page(cc_sample_api, test_url, page=pages, mime='text/html')
    raise AssertionError('Expected Failure')
except HTTPError as e:
    status = e.response.status_code
    if status != 400:
        raise AssertionError(f'Expected 400, got {status}')

## Downloading

In [ ]:
#export
from warcio import ArchiveIterator
from io import BytesIO

CC_DATA_URL = "https://commoncrawl.s3.amazonaws.com/"
def fetch_cc(filename: str, offset: int, length: int, session: Optional[Session] = None) -> bytes:
    if session is None:
        session = requests
    data_url = CC_DATA_URL + filename
    start_byte = int(offset)
    end_byte = start_byte + int(length)
    headers = {"Range": f"bytes={start_byte}-{end_byte}"}
    r = session.get(data_url, headers=headers)
    r.raise_for_status()
    
    # Decord WARC
    response_content = r.content
    archive = ArchiveIterator(BytesIO(response_content))
    record = next(archive)
    content = record.content_stream().read()
    
    # Archive should have just 1 record
    assert not any(True for _ in archive), "Expected 1 result in archive"
    
    return content

In [ ]:
#slow
record = records[0]
record

{'urlkey': 'org,wikipedia,mn)/wiki/%d0%81%d0%bb',
 'timestamp': '20211021123538',
 'url': 'https://mn.wikipedia.org/wiki/%D0%81%D0%BB',
 'mime': 'text/html',
 'mime-detected': 'text/html',
 'status': '200',
 'digest': 'UI6WL5QOXJAE26GCXG72FDYBINXIHKJA',
 'length': '17997',
 'offset': '515398132',
 'filename': 'crawl-data/CC-MAIN-2021-43/segments/1634323585405.74/warc/CC-MAIN-20211021102435-20211021132435-00353.warc.gz',
 'languages': 'mon,eng,srp',
 'encoding': 'UTF-8'}

In [ ]:
record = {'urlkey': 'org,wikipedia,en)/?curid=3516101',
 'timestamp': '20211024051554',
 'url': 'https://en.wikipedia.org/?curid=3516101',
 'mime': 'text/html',
 'mime-detected': 'text/html',
 'status': '200',
 'digest': 'TFZFWXWKS3NFJLLLXLYU6JTNZ77D3IVD',
 'length': '8922',
 'offset': '332329771',
 'filename': 'crawl-data/CC-MAIN-2021-43/segments/1634323585911.17/warc/CC-MAIN-20211024050128-20211024080128-00689.warc.gz',
 'languages': 'eng',
 'encoding': 'UTF-8'}

In [ ]:
%%time
content = fetch_cc(record['filename'], record['offset'], record['length'])

CPU times: user 27.8 ms, sys: 2.77 ms, total: 30.6 ms
Wall time: 1.13 s


In [ ]:
content[:1000]

b'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>Category:Tad Morose albums - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"362a73a3-175d-4af3-92bb-4707797d6067","wgCSPNonce":!1,"wgCanonicalNamespace":"Category","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":14,"wgPageName":"Category:Tad_Morose_albums","wgTitle":"Tad Morose albums","wgCurRevisionId":906041584,"wgRevisionId":906041584,"wgArticleId":3516101,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Set categories","Albums by artist","Power metal albums by Swedish artists"],"wgPageContentLanguage":"en","wgPageContentMode

Check the digest

In [ ]:
from hashlib import sha1
from base64 import b32encode

def get_digest(content: bytes) -> str:
    return b32encode(sha1(content).digest()).decode('ascii')

assert get_digest(content) == record['digest']

## Make it go fast

In [ ]:
#export
from joblib import delayed, Memory, Parallel
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

class CommonCrawl:
    def __init__(self, location: Optional[str],
                 session: Union[bool, Session] = True, threads: int=64, verbose=0) -> None:
        self.threads = threads
        self.memory = Memory(location, verbose=verbose)
        
        if session is True:
            session = make_session(pool_maxsize=threads)
        if not session:
            session = None
        self.session = session
        
        self._query_pages = self.memory.cache(query_cc_cdx_num_pages, ignore=['session'])
        self._query = self.memory.cache(query_cc_cdx_page, ignore=['session'])
        self._fetch = self.memory.cache(fetch_cc, ignore=['session'])
    
    @property
    def cdx_apis(self) -> Dict[str, str]:
        return {x['id']: x['cdx-api'] for x in get_cc_indexes()}
    
    # Note: don't use the default number of threads since we're hitting a single CC API
    # Todo several CC APIs
    def query(self, api: str, url: str, 
              start: Optional[str] = None, end: Optional[str] = None,
              status_ok: bool = True, mime: Optional[Union[str, Iterable[str]]] = None,
              limit: Optional[int] = None, offset: Optional[int] = None, force=False,
              threads: int=4) -> List[CaptureIndexRecord]:
        # lookup by key
        api = self.cdx_apis.get(api, api)
        query = self._query.call if force else self._query
        
        # Wrap this up?
        query_pages = self._query_pages.call if force else self._query_pages
        num_pages = query_pages(api, url, session=self.session)
        num_pages = num_pages[0] if force else num_pages
        
        pages_results = Parallel(threads, prefer='threads')(delayed(query)(api, url, page, start, end, status_ok, mime, limit, offset, session=self.session) for page in range(num_pages))
        if force:
            pages_results = [r[0] for r in pages_results]
        return [result for page in pages_results for result in page]
    
    def fetch_one(self, record, force=False) -> bytes:
        fetch = self._fetch.call if force else self._fetch
        result = fetch(record['filename'], record['offset'], record['length'], session=self.session)
        return result[0] if force else result
    
    def fetch(self, records, force=False, threads=None) -> List[bytes]:
        fetch = self._fetch.call if force else self._fetch
        result = Parallel(threads or self.threads, prefer='threads')(delayed(self.fetch_one)(record, force) for record in records)
        return result

### Test querying

In [ ]:
cc = CommonCrawl(test_cache, verbose=1)

In [ ]:
cc.memory.clear()

In [ ]:
assert 'CC-MAIN-2021-43' in cc.cdx_apis

In [ ]:
#slow
results = cc.query('CC-MAIN-2021-43', 'archive.org/s*', status_ok=False)


pd.DataFrame(results).groupby(['mime', 'status'])['url'].count()

________________________________________________________________________________
[Memory] Calling __main__--tmp-ipykernel-1284078378.query_cc_cdx_num_pages...
query_cc_cdx_num_pages('https://index.commoncrawl.org/CC-MAIN-2021-43-index', 'archive.org/s*', session=<requests.sessions.Session object at 0x7f06a60df910>)
___________________________________________query_cc_cdx_num_pages - 0.7s, 0.0min
________________________________________________________________________________
[Memory] Calling __main__--tmp-ipykernel-1284078378.query_cc_cdx_page...
query_cc_cdx_page('https://index.commoncrawl.org/CC-MAIN-2021-43-index', 'archive.org/s*', 1, None, None, False, None, None, None, session=<requests.sessions.Session object at 0x7f06a60df910>)
________________________________________________________________________________
[Memory] Calling __main__--tmp-ipykernel-1284078378.query_cc_cdx_page...
query_cc_cdx_page('https://index.commoncrawl.org/CC-MAIN-2021-43-index', 'archive.org/s*', 0, None, N

mime             status
audio/x-mpegurl  200         80
image/jpeg       200        134
image/x.djvu     302         26
text/html        200       7971
                 301       9543
                 302       6963
                 404        159
                 500          5
                 502          2
                 503          4
text/xml         200         20
Name: url, dtype: int64

In [ ]:
#slow
results2 = cc.query('CC-MAIN-2021-43', 'archive.org/s*', status_ok=False)

assert results == results2

In [ ]:
#slow
results3 = cc.query('CC-MAIN-2021-43', 'archive.org/s*', status_ok=False, force=True)

assert results == results3

________________________________________________________________________________
[Memory] Calling __main__--tmp-ipykernel-1284078378.query_cc_cdx_num_pages...
query_cc_cdx_num_pages('https://index.commoncrawl.org/CC-MAIN-2021-43-index', 'archive.org/s*', session=<requests.sessions.Session object at 0x7f06a60df910>)
___________________________________________query_cc_cdx_num_pages - 0.2s, 0.0min
________________________________________________________________________________
[Memory] Calling __main__--tmp-ipykernel-1284078378.query_cc_cdx_page...
query_cc_cdx_page('https://index.commoncrawl.org/CC-MAIN-2021-43-index', 'archive.org/s*', 1, None, None, False, None, None, None, session=<requests.sessions.Session object at 0x7f06a60df910>)
________________________________________________________________________________
[Memory] Calling __main__--tmp-ipykernel-1284078378.query_cc_cdx_page...
query_cc_cdx_page('https://index.commoncrawl.org/CC-MAIN-2021-43-index', 'archive.org/s*', 0, None, N

### Test status

In [ ]:
#slow
results_ok = cc.query('CC-MAIN-2021-43', 'archive.org/s*', status_ok=True)

assert results_ok == [r for r in results if r['status'] == '200']

________________________________________________________________________________
[Memory] Calling __main__--tmp-ipykernel-1284078378.query_cc_cdx_page...
query_cc_cdx_page('https://index.commoncrawl.org/CC-MAIN-2021-43-index', 'archive.org/s*', 0, None, None, True, None, None, None, session=<requests.sessions.Session object at 0x7f06a60df910>)
________________________________________________________________________________
[Memory] Calling __main__--tmp-ipykernel-1284078378.query_cc_cdx_page...
query_cc_cdx_page('https://index.commoncrawl.org/CC-MAIN-2021-43-index', 'archive.org/s*', 1, None, None, True, None, None, None, session=<requests.sessions.Session object at 0x7f06a60df910>)
________________________________________________query_cc_cdx_page - 0.8s, 0.0min
________________________________________________query_cc_cdx_page - 7.6s, 0.1min


### Test mimetypes

In [ ]:
#slow
results_html = cc.query('CC-MAIN-2021-43', 'archive.org/s*', mime='text/html', status_ok=False)

assert results_html == [r for r in results if r['mime'] == 'text/html']

________________________________________________________________________________
[Memory] Calling __main__--tmp-ipykernel-1284078378.query_cc_cdx_page...
query_cc_cdx_page('https://index.commoncrawl.org/CC-MAIN-2021-43-index', 'archive.org/s*', 0, None, None, False, 'text/html', None, None, session=<requests.sessions.Session object at 0x7f06a60df910>)
________________________________________________________________________________
[Memory] Calling __main__--tmp-ipykernel-1284078378.query_cc_cdx_page...
query_cc_cdx_page('https://index.commoncrawl.org/CC-MAIN-2021-43-index', 'archive.org/s*', 1, None, None, False, 'text/html', None, None, session=<requests.sessions.Session object at 0x7f06a60df910>)
_______________________________________________query_cc_cdx_page - 12.4s, 0.2min
_______________________________________________query_cc_cdx_page - 27.3s, 0.5min


In [ ]:
#slow
mimes = ['audio/x-mpegurl', 'image/x.djvu', 'text/xml']
results_mime = cc.query('CC-MAIN-2021-43', 'archive.org/s*', mime=mimes, status_ok=False)

assert results_mime == [r for r in results if r['mime'] in mimes]

________________________________________________________________________________
[Memory] Calling __main__--tmp-ipykernel-1284078378.query_cc_cdx_page...
query_cc_cdx_page('https://index.commoncrawl.org/CC-MAIN-2021-43-index', 'archive.org/s*', 0, None, None, False, ['audio/x-mpegurl', 'image/x.djvu', 'text/xml'], None, None, session=<requests.sessions.Session object at 0x7f06a60df910>)
________________________________________________________________________________
[Memory] Calling __main__--tmp-ipykernel-1284078378.query_cc_cdx_page...
query_cc_cdx_page('https://index.commoncrawl.org/CC-MAIN-2021-43-index', 'archive.org/s*', 1, None, None, False, ['audio/x-mpegurl', 'image/x.djvu', 'text/xml'], None, None, session=<requests.sessions.Session object at 0x7f06a60df910>)
________________________________________________query_cc_cdx_page - 4.6s, 0.1min
________________________________________________query_cc_cdx_page - 8.6s, 0.1min


In [ ]:
#slow
results_image = cc.query('CC-MAIN-2021-43', 'archive.org/s*', mime='image/*', status_ok=False)

assert results_image == [r for r in results if r['mime'].startswith('image/')]

________________________________________________________________________________
[Memory] Calling __main__--tmp-ipykernel-1284078378.query_cc_cdx_page...
query_cc_cdx_page('https://index.commoncrawl.org/CC-MAIN-2021-43-index', 'archive.org/s*', 0, None, None, False, 'image/*', None, None, session=<requests.sessions.Session object at 0x7f06a60df910>)
________________________________________________________________________________
[Memory] Calling __main__--tmp-ipykernel-1284078378.query_cc_cdx_page...
query_cc_cdx_page('https://index.commoncrawl.org/CC-MAIN-2021-43-index', 'archive.org/s*', 1, None, None, False, 'image/*', None, None, session=<requests.sessions.Session object at 0x7f06a60df910>)
________________________________________________query_cc_cdx_page - 5.1s, 0.1min
________________________________________________query_cc_cdx_page - 7.1s, 0.1min


### Check fetching

In [ ]:
cc = CommonCrawl(test_cache)

In [ ]:
result = {'urlkey': 'com,skeptric)/',
 'timestamp': '20211028110756',
 'url': 'https://skeptric.com/',
 'mime': 'text/html',
 'mime-detected': 'text/html',
 'status': '200',
 'digest': '7RBLUZ55MD4FUPDRPVVJVTM7YDYQXRS3',
 'length': '107238',
 'offset': '650789450',
 'filename': 'crawl-data/CC-MAIN-2021-43/segments/1634323588284.71/warc/CC-MAIN-20211028100619-20211028130619-00465.warc.gz',
 'languages': 'eng',
 'encoding': 'UTF-8'}

In [ ]:
contents = cc.fetch([result])
content = cc.fetch_one(result)
assert len(contents) == 1
assert content == contents[0]